In [1]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data from .npz file

In [4]:
cwd = os.getcwd()
loadpath = cwd + "/data/trainset.npz"
l = np.load(loadpath)
l
print (l.files)

['trainlabel', 'trainimg', 'testimg', 'testlabel']


In [5]:
# Load them!
cwd = os.getcwd()
loadpath = cwd + "/data/trainset.npz"
l = np.load(loadpath)

# See what's in here
print (l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
# use_gray = l['use_gray']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("%d classes" % (nclass))

['trainlabel', 'trainimg', 'testimg', 'testlabel']
69 train images loaded
18 test images loaded
900 dimensional input
2 classes


In [6]:
trainimg.shape

(69, 900)

In [7]:
trainlabel.shape

(69, 2)

# Network parameters

In [8]:
tf.set_random_seed(0)

#Parameter of logistic regression
learning_rate = 0.001
train_epochs = 1000
batch_size = 10
display_step = 100

#Create TF graph of logistic regression
x = tf.placeholder(dtype=tf.float32, shape=[None, dim])
y = tf.placeholder(dtype=tf.float32, shape=[None, nclass])

#Weights & biases
W = tf.Variable(tf.zeros([dim, nclass]), name='weights')
b = tf.Variable(tf.zeros([nclass]), name='biases')

# Define functions

In [9]:
WEIGHT_DECAY_FACTOR = 1 #0.000001
l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])

y_pred = tf.nn.softmax(tf.matmul(x, W) + b)
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred), reduction_indices=1))
loss = loss + WEIGHT_DECAY_FACTOR*l2_loss

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

init = tf.global_variables_initializer()

# Training phase

In [10]:
sess = tf.Session()
sess.run(init)

for epoch in range(train_epochs):
    avg_loss = 0
    num_batch = int(ntrain/batch_size)
    #Loop over all batches
    for i in range(num_batch):
        random_idx = np.random.randint(ntrain, size=batch_size)
        batch_x = trainimg[random_idx, :]
        batch_y = trainlabel[random_idx, :]
        
        #Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        
        #Compute avg_loss
        avg_loss += sess.run(loss, feed_dict={x: batch_x, y: batch_y}) / num_batch
        
    #Display logs per epoch step
    if epoch % display_step == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        test_accuracy = sess.run(accuracy, feed_dict={x: testimg, y: testlabel})
        
        print('Epoch: %d/%d, loss= %.4f\n     Train Accuracy: %.4f\n     Test Accuracy: %.4f' % (epoch, train_epochs, avg_loss, train_accuracy, test_accuracy))

Epoch: 0/1000, loss= 0.6800
     Train Accuracy: 0.5000
     Test Accuracy: 0.2778
Epoch: 100/1000, loss= 0.6238
     Train Accuracy: 0.9000
     Test Accuracy: 0.7222
Epoch: 200/1000, loss= 0.5297
     Train Accuracy: 0.9000
     Test Accuracy: 0.7222
Epoch: 300/1000, loss= 0.5253
     Train Accuracy: 1.0000
     Test Accuracy: 0.7778
Epoch: 400/1000, loss= 0.5806
     Train Accuracy: 0.8000
     Test Accuracy: 0.7778
Epoch: 500/1000, loss= 0.5085
     Train Accuracy: 0.9000
     Test Accuracy: 0.7778
Epoch: 600/1000, loss= 0.5377
     Train Accuracy: 0.9000
     Test Accuracy: 0.6667
Epoch: 700/1000, loss= 0.5686
     Train Accuracy: 0.9000
     Test Accuracy: 0.7222
Epoch: 800/1000, loss= 0.5344
     Train Accuracy: 0.9000
     Test Accuracy: 0.7778
Epoch: 900/1000, loss= 0.5302
     Train Accuracy: 1.0000
     Test Accuracy: 0.7778
